In [107]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

## Getting page content by Selenium

We need to load html content by selenium to avoid the Beautiful Soup load until the entire webpage is loaded.

In [108]:
URL = "https://velacat.sailti.com/ca/default/races/resultsajax/id/5047/idsc2r/15119/allResults/1/handicap"

In [109]:
COMPETICAO = "2021 ILCA Standard Men's World Championship"
CATEGORIA = "ILCA 7"

In [110]:
options = Options()
options.add_argument("--headless")

with Chrome(options=options) as browser:
    browser.get(URL)
    html = browser.page_source

In [111]:
soup = BeautifulSoup(html, "html.parser")

All data are in rows of a table, so we can find them by finding the tbody tag

In [112]:
for tr in soup.find("tbody"):
    print(tr)
    print("---")



---
<tr class="odd"><td align="center" class="position">1</td><td class="nowrap"><img src="/images/country_flags/NZL.gif"/>  NZL 218910</td><td></td><td>Thomas SAUNDERS<br/> </td><td></td><td></td><td align="center" class="netpoints">23</td><td align="center" class="totalpoints">60</td><td align="center" class="pos col-2x0" rel="2x0">
<div align="center" class="punt_Yellow">
<span style="display:none;">punt_Yellow _0000003</span> 3
                                            </div>
</td><td align="center" class="pos col-3x0" rel="3x0">
<div align="center" class="punt_Yellow">
<span style="display:none;">punt_Yellow _0000004</span> 4
                                            </div>
</td><td align="center" class="pos col-4x0" rel="4x0">
<div align="center" class="punt_Blue">
<span style="display:none;">punt_Blue _0000007</span> 7
                                            </div>
</td><td align="center" class="pos col-5x0" rel="5x0">
<div align="center" class="punt_Blue">
<span style

In [113]:
NUMERO_DE_PARTICIPANTES = 135
DADOS_POR_LINHA = 15

In [143]:
def get_flotilha(soup_object):
    """
    This functions gets a input like <span style="display:none;">punt_Yellow _0000003</span>
    and returns the series of this data.
    """
    if "Yellow" in str(soup_object) or "Blue" in str(soup_object):
        return "G"
    elif "Silver" in str(soup_object):
        return "P"
    elif "Gold" in str(soup_object):
        return "O"
    else:
        return "TO VERIFY"
    

In [144]:
def get_pontuacao(text):
    """
    This functions geta a string like "'\n\npunt_Gold _0000002 2\n                                            \n'"
    and extract the part referent to the pontuation
    """
    try:
        pontuacao = text.split("\n")[2].split(" ")[-1]
    except:
        pontuacao= "TO VERIFY"
    return pontuacao
    

In [145]:
data = {
    "Id Resultado": [],
    "Id Competidor": [],
    "Nome Competidor": [],
    "Id Competição": [],
    "Classe Vela": [],
    "Pontuação Regata": [],
    "Flotilha": [],
    "Posição Geral": [],
    "Punição": [],
    "Pontuação Total": [],
    "Nett": [],
    "Nome Competição": []    
}

In [146]:
linhas = soup.find("tbody").findChildren("td")
for i in range(0, (NUMERO_DE_PARTICIPANTES//DADOS_POR_LINHA)+1):
    for j in range(8, 16):
        data["Id Resultado"].append(None)
        data["Id Competidor"].append(None)
        data["Nome Competidor"].append(linhas[3+DADOS_POR_LINHA*i].text),
        data["Id Competição"].append(None),
        data["Classe Vela"].append(CATEGORIA),
        data["Pontuação Regata"].append(get_pontuacao(linhas[j + DADOS_POR_LINHA*i].text)),
        data["Posição Geral"].append(linhas[DADOS_POR_LINHA*i].text),
        data["Flotilha"].append(get_flotilha(linhas[j+DADOS_POR_LINHA*i]))
        data["Punição"].append(None),
        data["Pontuação Total"].append(linhas[7+DADOS_POR_LINHA*i].text),
        data["Nett"].append(linhas[6 * DADOS_POR_LINHA*i].text),
        data["Nome Competição"].append(COMPETICAO)
        


In [147]:
df= pd.DataFrame(data)

In [150]:
df.to_csv(f"results/{COMPETICAO+CATEGORIA}.csv")

In [59]:
linhas = soup.find("tbody").findChildren("td")

In [77]:
posicao = linhas[0].text
print(posicao)

1


In [78]:
nome = linhas[3].text
print(nome)

Thomas SAUNDERS 


In [79]:
net = linhas[6].text
print(net)

23


In [80]:
pontuacao = linhas[7].text
print(pontuacao)

60


In [ ]:
for i in range(8, 16): #

In [105]:
linhas[15].text

'\n\npunt_Gold _0000002 2\n                                            \n'

In [83]:
linhas[17].text

'  IRL 216890'

In [94]:
for i in range(8, 16):
    print(linhas[i].find("span"))

<span style="display:none;">punt_Yellow _0000003</span>
<span style="display:none;">punt_Yellow _0000004</span>
<span style="display:none;">punt_Blue _0000007</span>
<span style="display:none;">punt_Blue _0000002</span>
<span style="display:none;">punt_Blue _0000037</span>
<span style="display:none;">punt_Gold _0000001</span>
<span style="display:none;">punt_Gold _0000004</span>
<span style="display:none;">punt_Gold _0000002</span>


In [129]:
linhas[120]

<td align="center" class="pos col-2x0" rel="2x0">
<div align="center" class="punt_Blue">
<span style="display:none;">punt_Blue _0000005</span> 5
                                            </div>
</td>